In [62]:
pip install jupysql --upgrade

In [63]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [64]:
import duckdb
conn = duckdb.connect()
conn.query("INSTALL sqlite")
%config SqlMagic.displaylimit = 100

In [65]:
%sql duckdb:///data/basic_examples.db --alias duckdb

MissingPackageError: Can't load plugin: sqlalchemy.dialects:duckdb

To fix it, run this in your notebook: %pip install duckdb-engine --quiet

For more details, see: https://jupysql.ploomber.io/en/latest/howto/db-drivers.html#duckdb
If you need help solving this issue, send us a message: https://ploomber.io/community


In [ ]:
import os
if os.path.exists("/home/shared/sql/imdb_duck.db"):
  imdbpath = "duckdb:////home/shared/sql/imdb_duck.db"
elif os.path.exists("data/imdb_duck.db"):
  imdbpath = "duckdb:///data/imdb_duck.db"
else:
  import gdown
  url = 'https://drive.google.com/uc?id=...'
  output_path = 'data/imdb_duck.db'
  gdown.download(url, output_path, quiet=False)
  imdbpath = "duckdb:///data/imdb_duck.db"
print(imdbpath)

In [ ]:
from sqlalchemy import create_engine
imdb_engine = create_engine(imdbpath, connect_args={'read_only': True})
%sql imdb_engine --alias imdb

In [ ]:
%%sql duckdb
SELECT *
FROM Dragon
ORDER BY cute DESC;

In [ ]:
%%sql duckdb
SELECT *
FROM Dragon
LIMIT 2;

In [ ]:
%%sql duckdb
SELECT *
FROM Dragon
LIMIT 2
OFFSET 1;

In [ ]:
%%sql duckdb
SELECT *
FROM Dragon
ORDER BY RANDOM()
LIMIT 2

In [ ]:
%%sql duckdb
SELECT *
FROM Dragon USING SAMPLE reservoir(2 ROWS) REPEATABLE (100);

In [ ]:
%%sql duckdb
SELECT *
FROM Dish;

In [ ]:
%%sql duckdb
SELECT type
FROM Dish;

In [ ]:
%%sql duckdb
SELECT type
FROM Dish
GROUP BY type;

In [ ]:
%%sql duckdb
SELECT type, SUM(cost)
FROM Dish
GROUP BY type;

In [ ]:
%%sql duckdb
SELECT type,
  SUM(cost),
  MIN(cost),
  MAX(name)
FROM Dish
GROUP BY type;

In [ ]:
%%sql duckdb
SELECT year, COUNT(cute)
FROM Dragon
GROUP BY year;

In [ ]:
%%sql duckdb
SELECT year, COUNT(*)
FROM Dragon
GROUP BY year;

In [ ]:
%%sql duckdb

SELECT *
FROM Dish;

In [ ]:
%%sql duckdb

SELECT type, COUNT(*)
FROM Dish
GROUP BY type
HAVING MAX(cost) < 8;

In [ ]:
%%sql duckdb

SELECT type, COUNT(*)
FROM Dish
WHERE cost < 8
GROUP BY type

In [ ]:
%%sql duckdb

SELECT type, MAX(name)
FROM DishDietary
WHERE notes == 'gf'
GROUP BY type
HAVING MAX(cost) <= 7;

In [ ]:
%%sql imdb
SELECT setseed(0.42);

SELECT *
FROM Title
ORDER BY RANDOM()
LIMIT 10;

In [ ]:
%%sql imdb

SELECT titleType, primaryTitle
FROM Title
WHERE primaryTitle LIKE '%Star Wars%'

In [ ]:
%%sql imdb

SELECT titleType, primaryTitle
FROM Title
WHERE primaryTitle SIMILAR TO '.*Star Wars.*'

In [ ]:
%%sql imdb

SELECT titleType, primaryTitle
FROM Title
WHERE primaryTitle LIKE 'Harry Potter and the Deathly Hallows: Part _'

In [ ]:
%%sql imdb

SELECT primaryTitle, CAST(runtimeMinutes AS REAL)
FROM Title
LIMIT 10;

In [ ]:
%%sql imdb

SELECT setseed(0.42);

SELECT titleType, startYear,
CASE WHEN startYear < '1950' THEN 'old'
  WHEN startYear < '2000' THEN 'mid-aged'
  ELSE 'new'
  END AS movie_age
FROM Title
ORDER BY RANDOM()
LIMIT 10;

In [ ]:
%%sql duckdb
SELECT * FROM s;

In [ ]:
%%sql duckdb
SELECT * FROM t;

In [ ]:
%%sql duckdb
SELECT s.id, name, breed
FROM s INNER JOIN t ON s.id = t.id;

In [ ]:
%%sql duckdb
SELECT s.id, name, breed
FROM s JOIN t ON s.id = t.id;

In [ ]:
%%sql duckdb
SELECT *
FROM s CROSS JOIN t;

In [ ]:
%%sql duckdb
SELECT *
FROM s,t

In [ ]:
%%sql duckdb
SELECT * CROSS JOIN t
WHERE s.id = t.id;

In [ ]:
%%sql duckdb

SELECT *
FROM s LEFT JOIN t ON s.id = t.id;

In [ ]:
%%sql duckdb

SELECT *
FROM s RIGHT JOIN t ON s.id = t.id;

In [ ]:
%%sql duckdb

SELECT *
FROM s FULL JOIN t ON s.id = t.id;

In [ ]:
%%sql imdb

SELECT primaryTitle, averageRating, genres,
FROM Title AS T INNER JOIN Rating AS R
  ON T.tconst = R.tconst
WHERE numVotes > 100
ORDER BY averageRating DESC
LIMIT 10;

In [ ]:
%%sql imdb

SELECT primaryTitle, averageRating
FROM Title T INNER JOIN Rating R
  ON T.tconst = R.tconst
WHERE numVotes > 100
ORDER BY averageRating DESC
LIMIT 10;

In [ ]:
%%sql imdb

SELECT tbl_name, sql FROM sqlite_master WHERE type='table'

In [ ]:
%%sql imdb

SELECT *
FROM Title
WHERE primaryTitle IN ('Ginny & Georgia', 'What If...?', 'Succession', 'Veep', 'Tenet')
LIMIT 50;

In [ ]:
%%sql imdb

SELECT
  tconst AS id,
  primaryTitle AS title,
  titleType,
  runtimeMinutes,
  startYear
FROM Title
WHERE
  runtimeMinutes > 0 AND runtimeMinutes < 300
  AND startYear > 0
LIMIT 10

In [ ]:
%%sql imdb

SELECT
  T.tconst AS id,
  primaryTitle AS title,
  titleType,
  runtimeMinutes,
  startYear,
  genres,
  averageRating,
  numVotes
FROM Title T, Rating R
WHERE
  T.tconst = R.tconst
  AND runtimeMinutes > 0 AND runtimeMinutes < 300
  AND startYear > 0
  AND titleType = 'movie'
  AND numVotes >= 1000
  AND genres LIKE '%Action%'
ORDER BY averageRating DESC
LIMIT 10

In [ ]:
%%sql
WITH good_action_movies AS(
  SELECT *
  FROM Title T JOIN Rating R ON T.tconst = R.tconst
  WHERE genres LIKE '%Action%' AND averageRating > 7 AND numVotes > 5000
),
prolific_actors AS (
  SELECT N.nconst, primaryName, COUNT(*) as numRoles
  FROM Name N JOIN Principal P ON N.nconst = P.const
  WHERE category = 'actor'
  GROUP BY N.nconst, primaryName
)
SELECT primaryTitle, primaryName, numRoles, ROUND(averageRating) AS rating
FROM good_action_movies m, prolific_actors a, principal p
WHERE p.tconst = m.tconst AND p.nconst = a.nconst
ORDER BY rating DESC, numRoles DESC
LIMIT 10

In [ ]:
%%sql imdb
%config SqlMagic.autopandas = True

In [ ]:
%%sql action_movies_df <<
SELECT
  T.tconst AS id,
  primaryTitle AS title,
  titleType,
  runtimeMinutes AS time,
  startYear AS year,
  genres,
  averageRating AS rating,
  numVotes
FROM Title T, Rating R
WHERE
  T.tconst = R.tconst
  AND time > 0 AND time < 300
  AND year > 0
  AND titleType = 'movie'
  AND numVites >= 1000
  AND genres LIKE '%Action%'
ORDER BY startYear

In [ ]:
action_movies_df

In [ ]:
import plotly.express as px
px.scatter(action_movies_df, x = "year", y= "rating", opacity = 0.5)

In [ ]:
%%sql action_grouped_df <<

WITH action_movies AS (
  SELECT
    T.tconst AS id,
    primaryTitle AS title,
    titleType,
    runtimeMinutes AS time,
    startYear AS year,
    genres,
    averageRating AS rating,
    numVotes
  FROM Title T, Rating R
  WHERE
    T.tconst = R.tconst
    AND time > 0 AND time < 300
    AND year > 0
    AND titleType = 'movie'
    AND numVotes >= 1000
  AND genres LIKE '%Action%'
)
SELECT
  year,
  median(rating) AS median_rating,
  quantile_cont(rating, .25) AS lower_q,
  quantile_cont(rating, .75) AS upper_q,
FROM action_movies
GROUP BY year
ORDER BY year

In [ ]:
import plotly.express as px
action_grouped_df['error_p'] = action_grouped_df['upper_q'] - action_grouped_df['median_rating']
action_grouped_df['error_m'] = action_grouped_df['median_rating'] - action_grouped_df['lower_q']
fig = px.line(action_grouped_df, x="year", y="median_rating",
              error_y="error_p", error_y_minus="error_m")
fig

In [ ]:
px.scatter(action_movies_df, x="time", y="rating", opacity=0.1)

In [ ]:
px.density_heatmap(action_movies_df, x="time", y="rating", nbinsx=100, nbinsy=100)

In [ ]:
%%sql action_movies_df <<
WITH action_movies AS (
  SELECT
    T.tconst AS id,
    primaryTitle AS title,
    titleType,
    runtimeMinutes AS time,
    startYear AS year,
    genres,
    averageRating AS rating,
    numVotes
  FROM Title T, Rating R
  WHERE
    T.tconst = R.tconst
    AND time > 0 AND time < 300
    AND year > 0
    AND titleType = 'movie'
    AND numVotes >= 1000
  AND genres LIKE '%Action%'
)
SELECT
  ceil(time / 10) * 10 as time_desc,
  median(rating) AS median_rating,
  quantile_cont(rating, .25) AS lower_q,
  quantile_cont(rating, .75) AS upper_q,
FROM action_movies
GROUP BY time_desc
ORDER BY time_desc

In [ ]:
import plotly.express as px
action_grouped_df['error_p'] = action_grouped_df['upper_q'] - action_grouped_df['median_rating']
action_grouped_df['error_m'] = action_grouped_df['median_rating'] - action_grouped_df['lower_q']
fig = px.line(action_grouped_df, x="time_dexc", y="median_rating",
              error_y="error_p", error_y_minus="error_m")
fig

In [ ]:
import sklearn.linear_model as lm
import numpy as np
X, Y = action_grouped_df["time"].to_numpy(), action_grouped_df["rating"].to_numpy()

model = lm.LinearRegression()
model.fit(X[..., np.newaxis], Y)

xs = np.linspace(X.min(), X.max())
xhats = model.predict(xs[:, np.newaxis])

In [ ]:
fig = px.scatter(x=X, y=Y, opacity=0.1)
fig.add_scatter(x=xs, y=yhats)

In [ ]:
px.scatter(x=X, y=Y, opacity=0.1, trendline="ols")